# Notebook For Our Model

## Imports

In [56]:
import numpy as np
import math
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision.io import read_image, ImageReadMode
from torchvision.transforms import ToTensor
import glob
import matplotlib.pyplot as plot
from sklearn.metrics import confusion_matrix, accuracy_score
import cv2
import shutil
import csv
import torch.nn as nn
import torch.nn.functional as F

## Database

In [115]:
class XRAY(Dataset):
    def __init__(self, metadata, transform=None, target_transform=None):
        self.meta = metadata
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.meta)
    
    def __getitem__(self, index):
        img_path = self.meta[index][0]
        image = read_image(img_path, mode=ImageReadMode.GRAY).numpy()
        image = image.astype(np.float32)
        label = self.meta[index][1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label


### Read Meta Data from CSV File and Create Dataset

In [116]:
metadataPath = "./MetaData.csv"
metaData = []

with open(rf"{metadataPath}") as metadataFile:
    csv_reader = csv.reader(metadataFile)

    for row in csv_reader:
        if (row[1] == "Healthy"):
            metaData.append([row[0], 0])
        else:
            metaData.append([row[0], 1])


xrays = XRAY(metaData)

In [117]:
xrays[0][0]

array([[[202., 199., 195., ...,   5.,   2.,   0.],
        [199., 196., 195., ...,   5.,   2.,   0.],
        [196., 194., 193., ...,   5.,   2.,   0.],
        ...,
        [255., 255., 255., ...,   0.,   0.,   0.],
        [255., 255., 254., ...,   0.,   0.,   0.],
        [255., 255., 255., ...,   0.,   0.,   0.]]], dtype=float32)

In [122]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cnn_model = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels = 6, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=5),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=5),
            nn.Conv2d(in_channels=16, out_channels=30, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=5)
        )

        self.fc_model = nn.Sequential(
            nn.Linear(in_features=1470, out_features=500),
            nn.Tanh(),
            nn.Linear(in_features=500, out_features=100),
            nn.Tanh(),
            nn.Linear(in_features=100, out_features=1)
        )
    def forward(self, x):
        x = self.cnn_model(x)
        x = x.view(x.size(0), -1)
        x = self.fc_model(x)
        x = F.sigmoid(x)

        return x


In [123]:
device = torch.device('cuda:0')
model = Model().to(device)

In [124]:
dataloader = DataLoader(xrays, batch_size=32, shuffle=False)

In [126]:
model.eval()
outputs = []
y_true = []

maxBatches = 50
currentBatch = 0

with torch.no_grad():
    for D in dataloader:
        if (currentBatch > maxBatches):
            break
        image = D[0].to(device)
        label = D[1].to(device)
        
        y_hat = model(image)

        outputs.append(y_hat.cpu().detach().numpy())
        y_true.append(label.cpu().detach().numpy())
        currentBatch += 1